In [ ]:
# | echo: false
# | output: false

import sys
import os

sys.path.append(os.path.abspath("../../"))

%reload_ext autoreload
%autoreload 2

from python_utils.get_browser import get_browser_page_async

async def resolve_linkedin_username(uri: str) -> bool:
    page, close = await get_browser_page_async()
    response = None
    async def capture_response(resp):
        nonlocal response
        if uri in resp.url:
            response = resp
            await close()
    page.on("response", capture_response)
    await page.goto("https://www.linkedin.com/")
    await page.evaluate(f"""
    async () =>
        await fetch("{uri}", {{
                "mode": "no-cors",
                "credentials": "include"
            }})
    """)
    await close()
    return response.ok

class UsernameChecker:
    def __init__(self, username):
        self.username = username
        self.results = {}
        self.platforms = [
            # {
            #     "name": "LinkedIn Company",
            #     # can replace "www." with "de.", ".ke", ".ug", etc
            #     "uri": f"https://ke.linkedin.com/company/{username}",
            #     "resolve": resolve_linkedin_username,
            # },
            {
                "name": "LinkedIn User",
                # can replace "www." with "de.", ".ke", ".ug", etc
                "uri": f"https://ke.linkedin.com/in/{username}",
                "resolve": resolve_linkedin_username,
            }
        ]

    async def check_availability_async(self):
        for platform in self.platforms:
            try:
                is_available = await platform["resolve"](platform['uri'])
                if not is_available:
                    self.results[platform['name']] = {
                        'available': True,
                        'message': f"{self.username}: ✅ Available",
                        'url': platform['uri']
                    }
                else:
                    self.results[platform['name']] = {
                        'available': False,
                        'message': f"{self.username}: ❌ Taken",
                        'url': platform['uri']
                    }
            except Exception as e:
                self.results[platform['name']] = {
                    'available': None,
                    'error': str(e)
                }
        return self.results

In [64]:
# Create checker instance
checker = UsernameChecker("kamau")

# Check availability
await checker.check_availability_async()

{'LinkedIn User': {'available': True,
  'message': 'kamau: ✅ Available',
  'url': 'https://ke.linkedin.com/in/kamau'}}

In [67]:
username = "kamau"
url = f"https://ke.linkedin.com/in/{username}"
page, close = await get_browser_page_async()
response = None
async def capture_response(resp):
    global response
    if url in resp.url:
        response = resp
        await close()
page.on("response", capture_response)
await page.goto("https://www.linkedin.com/")
await page.evaluate(f"""
async () =>
    await fetch("{url}", {{
            "mode": "no-cors",
            "credentials": "include"
        }})
""")
f"{username}: ❌ Taken" if response.ok else f"{username}: ✅ Available", response.url

('kamau: ✅ Available', 'https://ke.linkedin.com/in/kamau')

In [ ]:
await response.text()

<coroutine object Response.text at 0xfd544c387ca0>